In [1]:
import os
import sys
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
from dask.distributed import Client

/home/ubuntu/lsst_stack_26/conda/envs/lsst-scipipe-7.0.1/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


In [2]:
client = Client(n_workers=8)

2024-02-29 13:52:31,894 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 01d1f9a3bfa7957dda8c51d449e24482 initialized by task ('shuffle-transfer-01d1f9a3bfa7957dda8c51d449e24482', 678) executed on worker tcp://127.0.0.1:33951
2024-02-29 13:55:51,890 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 01d1f9a3bfa7957dda8c51d449e24482 deactivated due to stimulus 'task-finished-1709214951.8882'


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 31.36 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44165,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.36 GiB
Comm: tcp://127.0.0.1:43471,Total threads: 1
Dashboard: http://127.0.0.1:37945/status,Memory: 3.92 GiB
Nanny: tcp://127.0.0.1:45527,


In [4]:
url_path = '/data/butler/dp02/dp02_products_datasets_urls.txt'

In [5]:
urls = dd.read_csv(url_path, header=None, dtype=str, names=['urls'])

In [6]:
urls.head()

,urls
0,https://ccnetlsst02.in2p3.fr:65105/2.2i/runs/F...
1,https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
2,https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
3,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
4,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...


In [7]:
# coadd_urls = urls['Coadd' in urls.urls.str]
object_urls = urls[urls['urls'].str.contains('objectTable', na=False)]

In [22]:
object_urls = object_urls.persist()

In [10]:
print(object_urls.head())

                                                     urls
17899   https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
27613   https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
42969   https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
51665   https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
144137  https://ccnetlsst02.in2p3.fr:65105/2.2i/runs/F...


In [11]:
object_urls['urls'].to_csv('/data/butler/dp02/object_urls.csv',single_file=True,header_first_partition_only=True)

['/data/butler/dp02/object_urls.csv']

In [12]:
def extract_directory(url,start,end):
    #trying to see what directories are there
    directories=url.split('/')[start:end]
    return '/'.join(directories)

In [14]:
object_urls['dir']= object_urls['urls'].apply(lambda x: extract_directory(x,10,11), meta=('str'))

In [16]:
object_urls['dir'].head()

17899     objectTable
27613     objectTable
42969     objectTable
51665     objectTable
144137    objectTable
Name: dir, dtype: object

In [18]:
directory_counts = object_urls['dir'].value_counts().compute()

In [19]:
directory_counts

dir
match_target_truth_summary_objectTable_tract     152
objectTable_tract                                157
match_ref_truth_summary_objectTable_tract        152
objectTable                                     7784
Name: count, dtype: int64

In [20]:
directory_counts_2 = directory_counts.reset_index()

In [29]:
object_urls = object_urls[object_urls['urls'].str.contains('/objectTable/')]

In [30]:
object_urls.head()

,urls,dir
17899,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...,objectTable
27613,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...,objectTable
42969,https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...,objectTable
51665,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...,objectTable
144137,https://ccnetlsst02.in2p3.fr:65105/2.2i/runs/F...,objectTable


In [31]:
object_urls.size.compute()

15568

In [32]:
object_urls = object_urls['urls']
object_urls.head()

17899     https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
27613     https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
42969     https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
51665     https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
144137    https://ccnetlsst02.in2p3.fr:65105/2.2i/runs/F...
Name: urls, dtype: object

In [33]:
object_urls['urls'].to_csv('/data/butler/dp02/object_urls_2.csv',single_file=True,header_first_partition_only=True)

['/data/butler/dp02/object_urls_2.csv']

In [101]:
client.close()